In [1]:
import numpy as np
import umbridge

In [ ]:
models = [
    umbridge.HTTPModel("http://localhost:4243", "posterior_fine"),
    umbridge.HTTPModel("http://localhost:4243", "posterior_intermediate"),
    umbridge.HTTPModel("http://localhost:4243", "posterior_coarse"),
]

model = models[2]
print(model.get_input_sizes())
print(model.get_output_sizes())

test_parameters = [[1, 1]]
model_output = model(test_parameters)

In [3]:
from models import posterior_pto_wrapper as wrapper

rng = np.random.default_rng(0)
noise = np.random.normal(0, 1, size=100)

parameter_ranges = np.array([[500, 5000], [1e5, 1e6], [5, 50], [5, 50]])
test_parameter_1 = np.array([1000, 5e5, 32, 17])
test_parameter_2 = np.array([10000, 5e5, 32, 17])

data = np.zeros((100,))
data[0:25] = test_parameter_1[0] * np.ones((25,))
data[25:50] = test_parameter_1[1] * np.ones((25,))
data[50:75] = test_parameter_1[2] * np.ones((25,))
data[75:100] = test_parameter_1[3] * np.ones((25,))
data += noise
covariance = np.eye(100)

models = [
    umbridge.HTTPModel("http://localhost:4243", "parameter_to_observable_map_fine"),
    umbridge.HTTPModel("http://localhost:4243", "parameter_to_observable_map_intermediate"),
    umbridge.HTTPModel("http://localhost:4243", "parameter_to_observable_map_coarse"),
]
model = models[2]

log_likelihood = wrapper.GaussianLogLikelihood(model, data, covariance)
log_prior = wrapper.UninformLogPrior(parameter_ranges)
log_posterior = wrapper.LogPosterior(log_prior, log_likelihood)
output = log_posterior(test_parameter_2)
np.isneginf(output[0][0])

True